In [9]:
import pygame
import neat
import numpy as np
import random
import os
from math import sin,cos,pi,sqrt

In [10]:
L = np.array([100,100,50])
v = np.array([0.75,0.75,1])
pygame.init()
font = pygame.font.Font('freesansbold.ttf',32)
win = pygame.display.set_mode((600,600))
win.fill ([255,255,255])
pygame.display.set_caption("Robotic Arm")

In [11]:
class Arm:
    def __init__(self,L,theta,v):
        self.L = L
        self.theta = -theta
        self.v =v
        self.DOF = len(L)
        self.dt = 0.1
    def Draw(self):
        x0  = 300
        y0  = 300
        for i in range(self.DOF):
            X = self.L[i]*cos(self.theta[i]) + x0 
            Y = self.L[i]*sin(self.theta[i]) + y0 
            pygame.draw.line(win,[250,0,0],(x0,y0),(X,Y),4)
            x0 = X
            y0 = Y
        return np.array([x0,y0])
    def GetHand(self):
        x0  = 300
        y0  = 300
        for i in range(self.DOF):
            X = self.L[i]*cos(self.theta[i]) + x0 
            Y = self.L[i]*sin(self.theta[i]) + y0 
            x0 = X
            y0 = Y
        return np.array([x0,y0])
    def Rl1(self,direction):
            if direction:
                self.theta[0]+= self.v[0]*self.dt
                self.theta[1]+= self.v[0]*self.dt
                self.theta[2]+= self.v[0]*self.dt
            else:
                self.theta[0]-= self.v[0]*self.dt
                self.theta[1]-= self.v[0]*self.dt
                self.theta[2]-= self.v[0]*self.dt
    def Rl2(self,direction):
        if direction:
            self.theta[1]+= self.v[1]*self.dt
            self.theta[2]+= self.v[1]*self.dt
        else:
            self.theta[1]-= self.v[1]*self.dt
            self.theta[2]-= self.v[1]*self.dt
    def Rl3(self,direction):
        if direction:
            self.theta[2]+= self.v[2]*self.dt 
        else:
            self.theta[2]-= self.v[2]*self.dt 
        

In [12]:
def Object():
    r = random.randint(0,251)
    o = random.random()*2*pi
    x = r*cos(o) +300
    y = r*sin(o) +300
    return np.array([x,y])


In [13]:
def DisplayFitness(value):
    score = font.render('fitness = '+str(value),True,[30,30,250])
    win.blit(score,[10,10])

In [14]:
def eval_genomes(genomes, config):
    nets = []
    arms = []
    ball = Object()
    for id, g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)
        nets.append(net)
        g.fitness = 0
        arm = Arm(L,(2*pi)*np.random.rand(3),v)
        arms.append(arm)
   
        
    running = True
    counter = 0
    while running:
        pygame.time.delay(5)
        pygame.display.update()
        for event in pygame.event.get():
             if event.type  == pygame.QUIT:
                running = False
                pygame.quit()
        
        win.fill ([255,255,255])        
        for i, arm in enumerate(arms):
            hand = arms[i].GetHand() 
            d1 = ball[0]-hand[0]
            d2 = ball[1]-hand[1]
            x1 = abs(ball[0]-hand[0])
            y1 = abs(ball[1]-hand[1])
            output = nets[i].activate((arms[i].theta[0],arms[i].theta[1],arms[i].theta[2],d1,d2))
            if output[0] > 0.75:
                arms[i].Rl3(True)
            if output[0] <- 0.75:
                arms[i].Rl3(False)
            if output[1] > 0.75:
                arms[i].Rl2(True)
            if output[1] <- 0.75:
                arms[i].Rl2(False)
            if output[2] > 0.75:
                arms[i].Rl1(True)
            if output[2] <- 0.75:
                arms[i].Rl1(False)            
            pygame.draw.circle(win,[30,30,250],[int(ball[0]),int(ball[1])],10,4) 
            hand = arms[i].Draw()   
            x2 = abs(ball[0]-hand[0])
            y2 = abs(ball[1]-hand[1])
            if (x2<x1 and y2<y1) or (x2<10 and y2<10):               
                genomes[i][1].fitness += 10
            else:
                genomes[i][1].fitness -= 100
        counter = counter+ 1
        if counter == 150:            
            running = False

        
        
    

In [15]:
def run(config_file):
 
    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)
    p = neat.Population(config)

    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    winner = p.run(eval_genomes, 1000000)


In [16]:
if __name__ == '__main__':
 
    #local_dir = os.path.dirname("__file__")
    config_path = os.path.join("C:\\Users\\Rayan\\Desktop\\Python\\Neat.txt")
    run(config_path)


 ****** Running generation 0 ****** 

Population's average fitness: -11128.00000 stdev: 1406.96837
Best fitness: -6970.00000 - size: (5, 16) - species 5 - id 24
Average adjusted fitness: 0.440
Mean genetic distance 2.934, standard deviation 0.607
Population of 27 members in 10 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0     1  -14120.0    0.206     0
     2    0     1  -11260.0    0.441     0
     3    0     2  -10600.0    0.515     0
     4    0    13  -13020.0    0.406     0
     5    0     1  -11810.0    0.542     0
     6    0     1  -14450.0    0.287     0
     7    0     2  -10380.0    0.544     0
     8    0     2  -11700.0    0.375     0
     9    0     2  -10160.0    0.574     0
    10    0     2  -11590.0    0.515     0
Total extinctions: 0
Generation time: 1.425 sec

 ****** Running generation 1 ****** 

Population's average fitness: -11284.44444 stdev: 1549.66982
Best fitness: -7850.00000 - size: (6, 17) - specie

error: display Surface quit